In [1]:
import paramiko

In [2]:
CMD = "tc qdisc change dev ens3 root netem delay {}ms"
JUMP_HOST_IP = "128.131.58.33"
NODES = {
    0: "128.131.58.126",
    1: "128.131.58.254",
    2: "128.131.58.204",
    3: "128.131.58.119",
}


In [3]:
def execute_on_node(node_id: int, cmd: str):
    proxy = paramiko.SSHClient()
    proxy.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    proxy.connect(JUMP_HOST_IP, 22, key_filename="/home/silv/.ssh/id_ed25519",
                  username="root")
    transport = proxy.get_transport()
    channel = transport.open_channel("direct-tcpip", (NODES[node_id], 22),
                                     ("localhost", 22))

    node = paramiko.SSHClient()
    node.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    node.connect(NODES[node_id], 22, key_filename="/home/silv/.ssh/id_ed25519",
                 username="root", sock=channel)
    _, stdout, stderr = node.exec_command(cmd)
    out = stdout.read().decode('utf8')
    err = stderr.read().decode('utf8')
    node.close()
    proxy.close()
    return out, err


def set_delay_for_node(node_id: int, delay: int):
    execute_on_node(node_id, CMD.format(delay))


def get_delay_for_node(node_id: int):
    stdout, stderr = execute_on_node(node_id, "tc qdisc show dev ens3")
    print(stdout)
    print(stderr)

In [4]:
node_latencies = [0, 0, 0, 0]

In [5]:
for node_id, latency in enumerate(node_latencies):
    set_delay_for_node(node_id, latency)

In [6]:
for node_id in range(4):
    get_delay_for_node(node_id)

qdisc netem 8002: root refcnt 2 limit 1000 delay 10ms

qdisc netem 8001: root refcnt 2 limit 1000 delay 30ms

qdisc netem 8001: root refcnt 2 limit 1000 delay 100ms

qdisc netem 8001: root refcnt 2 limit 1000 delay 300ms
